In [2]:
import pandas as pd
import re

base_path = '../data_brut/result'
all_dataframes = []

# Liste des noms de fichiers jusqu'en 2012
file_names_old_format = [
    'cdsp_presi1965t1_circ.csv', 'cdsp_presi1965t2_circ.csv',
    'cdsp_presi1969t1_circ.csv', 'cdsp_presi1969t2_circ.csv',
    'cdsp_presi1974t1_circ.csv', 'cdsp_presi1974t2_circ.csv',
    'cdsp_presi1981t1_circ.csv', 'cdsp_presi1981t2_circ.csv',
    'cdsp_presi1988t1_circ.csv', 'cdsp_presi1988t2_circ.csv',
    'cdsp_presi1995t1_circ.csv', 'cdsp_presi1995t2_circ.csv',
    'cdsp_presi2002t1_circ.csv', 'cdsp_presi2002t2_circ.csv',
    'cdsp_presi2007t1_circ.csv', 'cdsp_presi2007t2_circ.csv',
    'cdsp_presi2012t1_circ.csv', 'cdsp_presi2012t2_circ.csv',
]

# Liste des noms de fichiers de 2017 à 2022
file_names_new_format = [
    'cdsp_presi2017t1_circ.csv', 'cdsp_presi2017t2_circ.csv',
    'cdsp_presi2022t1_circ.csv', 'cdsp_presi2022t2_circ.csv',
]

for file_name in file_names_old_format + file_names_new_format:
    # Construire le chemin complet vers le fichier CSV
    csv_file_path = f"{base_path}/{file_name}"
    
    # Lire le fichier CSV dans un DataFrame
    df = pd.read_csv(csv_file_path, encoding='UTF-8', sep=',')


    # Utiliser une expression régulière pour trouver une année plausible dans le nom de fichier
    match = re.search(r'(\d{4})t\d', file_name)
    if match:
        year = int(match.group(1))  # Convertir l'année trouvée en entier
        df['Année'] = year
    else:
        year = None


    if file_name in file_names_new_format:
        # Rename columns 
        df.rename(columns={
            'Code du département': 'Code département',
            'Code de la circonscription': 'circonscription',
            'Libellé du département': 'département',
        }, inplace=True)
        # Upper columns Libellé du département


    # Convertir 'Code département' en entier de manière sûre
    df['Code département'] = pd.to_numeric(df['Code département'], errors='coerce').fillna(0).astype(int)
    # Convertir 'circonscription' en entier de manière sûre
    df['circonscription'] = pd.to_numeric(df['circonscription'], errors='coerce').fillna(0).astype(int)
    
    if year < 1986:
        circons_to_include = [1, 2]
    else:
        circons_to_include = [1, 2, 3]

    # Filtrer sur le département 34 et sur les circonscriptions 1 et 2
    if file_name in file_names_old_format:
        df_montpellier = df[(df['Code département'] == 34) & (df['circonscription'].isin(circons_to_include))]
    else:
        df_montpellier = df[(df['Code département'] == 34) & (df['circonscription'].isin(circons_to_include))]

    # Ajouter le DataFrame filtré à la liste
    all_dataframes.append(df_montpellier)

# Concaténer tous les dataframes filtrés en un seul dataframe
montpellier_data = pd.concat(all_dataframes)

# Afficher un aperçu du dataframe résultant
print(montpellier_data.head())  # Cela affiche les premières lignes du DataFrame final

# Sauvegarder le dataframe dans un fichier Parquet
montpellier_data.to_parquet('../dataset/montpellier_data.parquet')

#replace
montpellier_data.to_csv('../dataset/montpellier_data.csv', index=False)


     Code département département  circonscription  Inscrits  Votants  \
132                34     HERAULT                1   76316.0  65013.0   
133                34     HERAULT                2   69715.0  59236.0   
132                34     HERAULT                1   76310.0  63961.0   
133                34     HERAULT                2   69688.0  59111.0   
132                34     HERAULT                1   81097.0  61451.0   

     Exprimés  Blancs et nuls  MITTERRAND (CIR)  LECANUET (MRP)  \
132   64503.0           510.0           22376.0          8471.0   
133   58739.0           497.0           22309.0          8396.0   
132   61629.0          2332.0           34479.0             NaN   
133   57026.0          2085.0           32658.0             NaN   
132   60823.0           628.0               NaN             NaN   

     DE GAULLE (UNR)  ...  Jean LASSALLE  Philippe POUTOU  \
132          21861.0  ...            NaN              NaN   
133          19381.0  ...           